## 第四節　FF暖房機

1．適用範囲～4．記号及び単位は省略

### 5．最大暖房出力

$$\large
Q_{max,H,d,t} = q_{max,H} \times 3600 \times 10^{-6}
$$

$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h)  
$q_{max,H}$：最大暖房能力(W)

### 6．暖房エネルギー消費量

#### 6.1　消費電力量

$$
\large
E_{E,H,d,t} = \left\{ \begin{array}{ll}
    P_{rtd,H} \times \frac{Q_{T,H,d,t}}{Q_{max,H,d,t}} \times 10^{-3} & \small(Q_{T,H,d,t} \ge q_{min,H} \times 3600 \times 10^{-6}の場合)\\ 
    \left(P_{rtd,H} \times \frac{Q_{T,H,d,t}}{Q_{max,H,d,t}} + P_{itm,H} \right) \times 10^{-3} & \small(Q_{T,H,d,t} \lt q_{min,H} \times 3600 \times 10^{-6}の場合)\\ 
    \end{array}
\right.
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$q_{min,H}$：連続運転時最小能力(W)  
$P_{rtd,H}$：定格暖房消費電力(W)  
$P_{itm,H}$：断続時消費電力(W)  
$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの処理暖房負荷(MJ/h)  
$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h)  

日付$d$の時刻$t$における1時間当たりの処理暖房負荷$Q_{T,H,d,t}$が 0 の場合、$E_{E,H,d,t} = 0$とする。

#### 6.2　ガス消費量

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,H,d,t}$は、FF暖房機の燃料がガスの場合日付$d$の時刻$t$における１時間当たりの燃料消費量$E_{F,H,d,t}$に等しいとし、FF暖房機の燃料が灯油の場合0とする。

#### 6.3　灯油消費量

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,H,d,t}$は、FF暖房機の燃料が灯油の場合日付$d$の時刻$t$における１時間当たりの燃料消費量$E_{F,H,d,t}$に等しいとし、FF暖房機の燃料がガスの場合0とする。

#### 6.4　その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,H,d,t}$は、0とする。

### 7．燃料消費量

$$\large
E_{F,H,d,t} = \frac{Q_{T,H,d,t}}{e_{rtd,H}}
$$

$E_{F,H,d,t}$：日付$d$の時刻$t$における1時間当たりの燃料消費量(MJ/h)  
$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの処理暖房負荷(MJ/h)  
$e_{rtd,H}$：定格燃焼効率  

## 付録 A 機器の性能を表す仕様の決定方法

### A.1　記号及び単位

省略

### A.2　最大暖房能力

$$\large
q_{max,H} = 240.1 \times A_{HCZ}
$$

$q_{max,H}$：最大暖房能力(W)  
$A_{HCZ}$：FF 暖房機により暖房される暖冷房区画の床面積(m<sup>2</sup>)  

### A.3　連続運転時最小能力

連続運転時最小能力$q_{min,H}$が0を下回る場合は0とする。

$$\large
q_{min,H} = 0.4334 \times q_{max,H} - 540.1
$$

$q_{min,H}$：連続運転時最小能力(W)

### A.4　定格燃焼効率

省略

### A.5　定格暖房消費電力

$$\large
P_{rtd,H} = 3.13 \times 10^{-3} \times q_{max,H}
$$

$P_{rtd,H}$：定格暖房消費電力(W)

### A.6　断続時消費電力

断続時消費電力$P_{itm,H}$は、40.0Wとする。

### A.7　複数のFF暖房機が設置される場合の仕様の決定方法

省略